# Predicting the Reasons for Retraction: An Abstract-Based Classification System for Author Error and Fraudulent Research.

*by Md Fazle Rasul, December, 2023* 

## Introduction

I am interested in learning about large language models. In my project, I have used the BigBird model which is a sparse attention mechanism capable of handling long sequences. I have also used the Reformer model. Ultimately, I am performing a classification task. As a classifier, I have tried logistic regression and neural networks. The reason for my interest in large language models is the recent advancement in this field. Models like GPT-4 with their amazing capabilities had an instant global effect. That's why I chose to work with a large language model.

Retraction of a published paper means that the original paper should not have been published and its data or conclusion should not be used in other research. The goal of this project is to create a model that takes the abstract of a retracted paper and outputs either 0 (indicating retraction for author error) or 1 (indicating retraction for fraud). This project seems interesting to me because it might be a way of early detection of potential issues like if we have contradictory sentences in our abstract or if we claim something in our abstract that is way too good to be true. This project can also be thought of as a way of knowing about the transparency of the retraction process. I used the pre-trained BigBird model and the pre-trained reformer model to generate features by feeding the abstracts. And then I fed the features to the classifier to predict classes. As classifiers, I have tried the logistic regression model and neural networks.

## Methods

I obtained my dataset from the Retraction Watch Database [The Retraction Watch Database]. The Retraction Watch Database provides various reasons for the retraction of a paper. They chose PubMed as their initial source which is a publicly available database maintained by the U.S. National Institutes of Health. Additionally, they collected retracted articles from public access databases like Google Scholar, publisher sites like Springerlink, and ScienceDirect, journals like Nature, Science, The Journal of Biological Chemistry, institutional reports of research misconduct investigations, social media sites, and tips sent in by blog followers. The Retraction Watch staff constructed the reasons column based on the tags used in the Retraction Watch blog post, institutional misconduct findings, and information they have gained through investigations.  The error column which indicates the classes has been constructed in the dataset based on the reasons provided by the Retraction Watch Database. The reasons indicate three possible categories author error, fraud, and no clear determination. In cases where the reasons indicate author error and fraud, it has been classified as fraud. I have dropped all the samples for this project where no clear determination can be made. I also dropped the samples that had missing values for the abstract column. Finally, I had 13279 abstracts to work with among which 9135 samples were of class "fraud" and 4144 were of class "author error".

First, I planned to use the BERT model. However, I discovered that the BERT model can only handle sequences up to 512 tokens long. When I tokenized the abstracts with pre-trained BertTokenizer I found out that the number of tokens for most of the sequences is way more than 512. So, I had to pick models that could handle longer sequences. That's why I used the BigBird model that can handle up to 4096 tokens. The BigBird model is a sparse attention mechanism. That means, unlike BERT where each token attends to every other token in the sequence, in BigBird each token attends to a limited number of other tokens. More specifically, each token attends to some random tokens, sliding tokens, and global tokens [Understanding BigBird's block sparse attention]. Global tokens are tokens that attend every other token and are attended by all other tokens. Sliding tokens mean neighboring past and future tokens. Random tokens, as the name suggests, are randomly chosen tokens. This gives BigBird the capability to handle sequences almost eight times longer than BERT's maximum sequence length [Zaheer, Manzil, et al., 2020]. So, we can say BigBird's attention is kind of an approximation of BERT's  attention.  The BigBird model has 12 encoder layers in it. The number of attention heads for each attention layer in the transformer encoder is also 12 [BigBird].  I also used the reformer model that can handle even longer sequences. The reformer model uses Local Sensitive Hashing (LSH) attention instead of traditional attention which helps it avoid calculating the full product query key in the attention layers [Reformer]. The LSH attention is also an approximation of the normal full self-attention. The dimensionality of the projected key, query, and value vectors for the reformer model is 64. The number of attention heads for each attention layer in the transformer encoder is 12 which is similar to the BigBird model. I used the pre-trained AutoTokenizer class to tokenize the abstracts and add special tokens like CLS and SEP. After tokenization with this new tokenizer, there were just four abstracts for which the number of tokens was more than 4096. I dropped those four samples from my dataset. The list called tokenized contains all the abstracts where each abstract was in the form of a list of tokens. To process them all as a batch, I had to pad them up to the same size so that I could represent the inputs as a single 2D array. But if I directly send this to the BigBird model, it would be slightly confusing for it. So, I created another variable attention_mask to tell it to ignore the padding I had just added at the time of processing the inputs. I first tried to run the entire input set as one batch. But my kernel crashed saying it is running out of memory. The BigBird model needs a huge amount of memory to do its processing. I tried different batch sizes. A batch size of just ten samples also caused my kernel to crash. I finally decided to use a batch size of five samples.  After running the model I don't need the entire output. I added the CLS token at the beginning of each abstract earlier. Output for this token represents embedding for the entire abstract. It does not capture every bit of detail but it is more like a summarized representation of the abstract. Another important point is that I can not take the whole output because then there will be a huge number of features and I only have 13279 samples in my dataset. So, I sliced the part of the output that corresponds to the CLS token for each abstract. I saved them into a new variable called features. Each abstract is now stored in the form of a vector of size 768. So, the shape of features is (13279, 768). I split my dataset into train and test sets. Then, I trained my classifiers - first a logistic regression classifier and then a neural networks classifier. I used Scikit Learn's logistic regression classifier and its built-in methods. I varied the architecture of my neural network to find out what works best for this problem. I tried with no hidden layers first. I also tried with just one hidden layer with a different number of units each time. Then I tried with two hidden layers of different sizes and recorded the results.

## Results

The logistic regression classifier achieved an accuracy of 73.4% on the test set. As my dataset was not evenly balanced, I also determined the precision, recall, and F1 score. I used sklearn's precision_recall_fscore_support method to see these metrics for each class. 

| | Fraud | author_error |
|:-:|:-:|:-:|
| Precision | 0.768 | 0.679 |
| Recall | 0.936 | 0.342 |
| F1 score | 0.844 | 0.459 | ---

The model is performing very poorly for the author error class compared to its fraud class performance. The low recall score for the author error class might be due to the imbalanced dataset. I tried with neural network classifier also. It performed worse than the logistic regression model. I started with a network with no hidden layer and also tried some bigger networks but it could not match the logistic regression classifier's performance. I tried different combinations of epochs and learning rates. Whenever I tried a bigger network or more epochs (even with a small learning rate) the model overfitted. The best accuracy I could manage with a neural network is 72.2 on test data. That neural network consisted of two hidden layers. There were 20 units in the first layer and just 10 units in the second layer. I trained my neural network model for 500 epochs with a learning rate 0.001. To get a better idea about my model's performance I compared it with a dummy classifier. I used scikit learn's dummy classifier. It achieved an accuracy of 69.3% which is close to our model's performance. Precision, recall, and F1 score for the dummy classifier are presented in the table below:

| | Fraud | author_error |
|:-:|:-:|:-:|
| Precision | 0.7 | 0 |
| Recall | 1 | 0 |
| F1 score | 0.823 | 0 |
---
When I  used the reformer model to extract the features and later fed those features into a logistic regression classifier, I got only 67.7% accuracy which was even lower than the dummy classifier's accuracy. So, the BigBird model used as the feature generator, together with the logistic regression classifier achieved the best results which are slightly better but not significantly better in comparison to the dummy classifier's results.

## Conclusions

I concluded that only the abstract does not provide enough context to say whether the retraction was due to author error or fraud. So, I would like to try with abstract combined with the conclusion to see if it can increase the accuracy. Another thing is that my dataset has almost twice as many samples for the fraud class in comparison to the author error class. So, the models might be biased to predict test samples as fraud. In the future, I would like to try ensemble methods too. 

In this project, I learned about the BigBird and the Reformer models, their architecture, attention layers, and how these models handle long sequences. I learned about Scikit Learn's built-in methods to easily create the logistic regression classifier and calculate accuracy, precision, recall, and F1 score. The most challenging part of this project was to handle very long sequences. I first used the BERT model. The BERT model could not handle long abstracts, so I had to switch to the BigBird model. However, the problem is the BigBird model does not use full attention layers so the output for the CLS token has a slightly less accurate summary of the entire abstract compared to BERT's output of the CLS token. The BigBird model has also a limitation on the number of tokens which is 4096 but it is almost eight times BERT's limitation of 512 which kind of forced me to choose BigBird over the BERT model. The reformer model can handle much longer sequences but performs worse. The reason behind the reformer model's failure in this case might be due to its way of approximating full attention layers. So, the output generated for the CLS token in the case of the reformer model is less accurate than it is for the BigBird model. However, all these models need a huge amount of memory and a long time to train which makes it very difficult to work with them. The dummy classifier performing better than the reformer model with logistic regression was surprising for me in this project. I enjoyed doing the project. It was a great learning experience for me.



### References

* The Retraction Watch Database [Internet]. New York: The Center for Scientific Integrity. 2018. ISSN: 2692-465X. [Cited (27 October)]. Available from: http://retractiondatabase.org/.
* Understanding BigBird’s block sparse attention. Hugging Face – The AI community building the future. (n.d.). https://huggingface.co/blog/big-bird
* BigBird. (n.d.). https://huggingface.co/docs/transformers/model_doc/big_bird#transformers.BigBirdConfig 
* Zaheer, M., Guruganesh, G., Dubey, K. A., Ainslie, J., Alberti, C., Ontanon, S., ... & Ahmed, A. (2020). Big bird: Transformers for longer sequences. Advances in neural information processing systems, 33, 17283-17297.
* Reformer. (n.d.). https://huggingface.co/docs/transformers/model_doc/reformer#overview

In [1]:
import io
import nbformat
import glob
nbfile = glob.glob('Final Report.ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[0], 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, nbformat.NO_CONVERT)
word_count = 0
for cell in nb.cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[0], 'is', word_count)

Word count for file Final Report.ipynb is 2045
